In [1]:
import os
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

# assign to first GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#VRAM memory on-demand usage(prevent Keras takes all memory at the beginning)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True   
sess = tf.Session(config=config)

KTF.set_session(sess)


Using TensorFlow backend.


In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import regularizers
from keras.callbacks import LearningRateScheduler
from keras.utils import np_utils
from keras import regularizers

import numpy as np

In [3]:
num_classes = 10
num_predictions = 20
batch_size = 64

(x_main, y_main), (x_test, y_test) = cifar10.load_data()

print('x shape:', x_main.shape)
x_train=x_main[:40000]
y_train=y_main[:40000]
print('x_train shape:', x_train.shape)
x_val=x_main[40000:]
y_val=y_main[40000:]
print('x_val shape:', x_val.shape)
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)

x_test = x_test.astype('float32')
x_test /= 255
y_test = keras.utils.to_categorical(y_test, num_classes)

x shape: (50000, 32, 32, 3)
x_train shape: (40000, 32, 32, 3)
x_val shape: (10000, 32, 32, 3)


In [4]:
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
__________

In [ ]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003        
    return lrate

In [ ]:
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

#training


opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=125,\
                    verbose=1,validation_data=(x_val,y_val),callbacks=[LearningRateScheduler(lr_schedule)])

#save to disk
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')    

#testing
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

Epoch 1/125
625/625 [==============================] - 16s 26ms/step - loss: 1.9530 - acc: 0.4106 - val_loss: 1.6525 - val_acc: 0.4808
Epoch 2/125
625/625 [==============================] - 14s 23ms/step - loss: 1.4704 - acc: 0.5595 - val_loss: 1.2241 - val_acc: 0.6143
Epoch 3/125
625/625 [==============================] - 15s 24ms/step - loss: 1.2844 - acc: 0.6208 - val_loss: 1.1017 - val_acc: 0.6450
Epoch 4/125
625/625 [==============================] - 15s 24ms/step - loss: 1.1841 - acc: 0.6580 - val_loss: 1.0701 - val_acc: 0.6868
Epoch 5/125
625/625 [==============================] - 15s 24ms/step - loss: 1.1210 - acc: 0.6838 - val_loss: 0.9826 - val_acc: 0.7161
Epoch 6/125
625/625 [==============================] - 15s 24ms/step - loss: 1.0778 - acc: 0.7030 - val_loss: 0.9592 - val_acc: 0.7019
Epoch 7/125
625/625 [==============================] - 15s 24ms/step - loss: 1.0616 - acc: 0.7118 - val_loss: 0.9918 - val_acc: 0.7231
Epoch 8/125
625/625 [==============================] - 

625/625 [==============================] - 15s 24ms/step - loss: 0.7637 - acc: 0.8254 - val_loss: 0.8532 - val_acc: 0.8142
Epoch 62/125
625/625 [==============================] - 15s 24ms/step - loss: 0.7417 - acc: 0.8262 - val_loss: 0.7192 - val_acc: 0.8322
Epoch 63/125
625/625 [==============================] - 15s 24ms/step - loss: 0.7540 - acc: 0.8237 - val_loss: 0.6974 - val_acc: 0.8414
Epoch 64/125
625/625 [==============================] - 15s 24ms/step - loss: 0.7358 - acc: 0.8242 - val_loss: 0.7396 - val_acc: 0.8258
Epoch 65/125
625/625 [==============================] - 15s 24ms/step - loss: 0.7418 - acc: 0.8249 - val_loss: 0.8162 - val_acc: 0.7962
Epoch 66/125
625/625 [==============================] - 15s 24ms/step - loss: 0.7311 - acc: 0.8284 - val_loss: 0.7375 - val_acc: 0.8336
Epoch 67/125
625/625 [==============================] - 15s 24ms/step - loss: 0.7287 - acc: 0.8256 - val_loss: 0.8054 - val_acc: 0.8038
Epoch 68/125
377/625 [=================>............] - ETA: 